# importing the libraries

In [1]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Loading the dataset

In [2]:
mov_data=pd.read_csv('movies.csv')
mov_data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [3]:
mov_data.shape

(4803, 24)

In [4]:
mov_data.describe()

,index,budget,id,popularity,revenue,runtime,vote_average,vote_count
count,4803.000000,4.803000e+03,4803.000000,4803.000000,4.803000e+03,4801.000000,4803.000000,4803.000000
mean,2401.000000,2.904504e+07,57165.484281,21.492301,8.226064e+07,106.875859,6.092172,690.217989
std,1386.651002,4.072239e+07,88694.614033,31.816650,1.628571e+08,22.611935,1.194612,1234.585891
min,0.000000,0.000000e+00,5.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,1200.500000,7.900000e+05,9014.500000,4.668070,0.000000e+00,94.000000,5.600000,54.000000
50%,2401.000000,1.500000e+07,14629.000000,12.921594,1.917000e+07,103.000000,6.200000,235.000000
75%,3601.500000,4.000000e+07,58610.500000,28.313505,9.291719e+07,118.000000,6.800000,737.000000
max,4802.000000,3.800000e+08,459488.000000,875.581305,2.787965e+09,338.000000,10.000000,13752.000000


In [5]:
mov_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 4803 non-null   int64  
 1   budget                4803 non-null   int64  
 2   genres                4775 non-null   object 
 3   homepage              1712 non-null   object 
 4   id                    4803 non-null   int64  
 5   keywords              4391 non-null   object 
 6   original_language     4803 non-null   object 
 7   original_title        4803 non-null   object 
 8   overview              4800 non-null   object 
 9   popularity            4803 non-null   float64
 10  production_companies  4803 non-null   object 
 11  production_countries  4803 non-null   object 
 12  release_date          4802 non-null   object 
 13  revenue               4803 non-null   int64  
 14  runtime               4801 non-null   float64
 15  spoken_languages     

# Finding out missing values

In [6]:
mov_data.isnull().sum()

index                      0
budget                     0
genres                    28
homepage                3091
id                         0
keywords                 412
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
cast                      43
crew                       0
director                  30
dtype: int64

selecting only required columns

In [7]:
selected_features=['genres','keywords','tagline','cast','director']
print(selected_features)

['genres', 'keywords', 'tagline', 'cast', 'director']


In [8]:
# replacing  the null values with null string

for feature in selected_features:
    mov_data[feature]=mov_data[feature].fillna('')

In [9]:
mov_data['keywords'].isnull().sum()

0

In [10]:
# combining all the 5 columns

combined_features=mov_data['genres']+' '+mov_data['keywords']+' '+mov_data['tagline']+' '+mov_data['cast']+' '+mov_data['director']

In [11]:
combined_features

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  A newlywed couple's honeymoon ...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      A New Yorker in Shanghai Daniel Henney Eliza...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object

In [12]:
# converting the text into numerical data

vectorizer = TfidfVectorizer()


In [13]:
feature_vector=vectorizer.fit_transform(combined_features)
print(feature_vector)

  (0, 2432)	0.17272411194153
  (0, 7755)	0.1128035714854756
  (0, 13024)	0.1942362060108871
  (0, 10229)	0.16058685400095302
  (0, 8756)	0.22709015857011816
  (0, 14608)	0.15150672398763912
  (0, 16668)	0.19843263965100372
  (0, 14064)	0.20596090415084142
  (0, 13319)	0.2177470539412484
  (0, 17290)	0.20197912553916567
  (0, 17007)	0.23643326319898797
  (0, 13349)	0.15021264094167086
  (0, 11503)	0.27211310056983656
  (0, 11192)	0.09049319826481456
  (0, 16998)	0.1282126322850579
  (0, 15261)	0.07095833561276566
  (0, 4945)	0.24025852494110758
  (0, 14271)	0.21392179219912877
  (0, 3225)	0.24960162956997736
  (0, 16587)	0.12549432354918996
  (0, 14378)	0.33962752210959823
  (0, 5836)	0.1646750903586285
  (0, 3065)	0.22208377802661425
  (0, 3678)	0.21392179219912877
  (0, 5437)	0.1036413987316636
  :	:
  (4801, 17266)	0.2886098184932947
  (4801, 4835)	0.24713765026963996
  (4801, 403)	0.17727585190343226
  (4801, 6935)	0.2886098184932947
  (4801, 11663)	0.21557500762727902
  (4801, 1672

In [14]:
# measuring the similarity 

similarity= cosine_similarity(feature_vector)
print(similarity)

[[1.         0.07219487 0.037733   ... 0.         0.         0.        ]
 [0.07219487 1.         0.03281499 ... 0.03575545 0.         0.        ]
 [0.037733   0.03281499 1.         ... 0.         0.05389661 0.        ]
 ...
 [0.         0.03575545 0.         ... 1.         0.         0.02651502]
 [0.         0.         0.05389661 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.02651502 0.         1.        ]]


In [15]:
similarity.shape

(4803, 4803)

In [16]:
# getting the movie name from user

movie_name=input("Enter the fav movie name : ")

Enter the fav movie name : titanic


In [17]:
# creating a list with all movie names

list_all_titles=mov_data['title'].to_list()

In [18]:
list_all_titles

['Avatar',
 "Pirates of the Caribbean: At World's End",
 'Spectre',
 'The Dark Knight Rises',
 'John Carter',
 'Spider-Man 3',
 'Tangled',
 'Avengers: Age of Ultron',
 'Harry Potter and the Half-Blood Prince',
 'Batman v Superman: Dawn of Justice',
 'Superman Returns',
 'Quantum of Solace',
 "Pirates of the Caribbean: Dead Man's Chest",
 'The Lone Ranger',
 'Man of Steel',
 'The Chronicles of Narnia: Prince Caspian',
 'The Avengers',
 'Pirates of the Caribbean: On Stranger Tides',
 'Men in Black 3',
 'The Hobbit: The Battle of the Five Armies',
 'The Amazing Spider-Man',
 'Robin Hood',
 'The Hobbit: The Desolation of Smaug',
 'The Golden Compass',
 'King Kong',
 'Titanic',
 'Captain America: Civil War',
 'Battleship',
 'Jurassic World',
 'Skyfall',
 'Spider-Man 2',
 'Iron Man 3',
 'Alice in Wonderland',
 'X-Men: The Last Stand',
 'Monsters University',
 'Transformers: Revenge of the Fallen',
 'Transformers: Age of Extinction',
 'Oz: The Great and Powerful',
 'The Amazing Spider-Man 2',

In [19]:
# finding closest match

find_match = difflib.get_close_matches(movie_name,list_all_titles)
print(find_match)

['Titanic', 'Maniac', 'Hitman']


In [20]:
# still close match

close_match=find_match[0]
print(close_match)

Titanic


In [21]:
# finding index of the movie with title

index_movie=mov_data[mov_data.title == close_match]['index'].values[0]

print(index_movie)

25


In [22]:
# getting list of similar movies

similarity_score= list(enumerate(similarity[index_movie]))
similarity_score

[(0, 0.042849260959502256),
 (1, 0.0),
 (2, 0.011467364105495693),
 (3, 0.010046067299207014),
 (4, 0.010913900929666012),
 (5, 0.013271868195005245),
 (6, 0.0),
 (7, 0.044276351205847934),
 (8, 0.0),
 (9, 0.011052967674011078),
 (10, 0.040774737457045525),
 (11, 0.005154880168301485),
 (12, 0.0),
 (13, 0.011810642378201048),
 (14, 0.011135108727728622),
 (15, 0.010700980101090706),
 (16, 0.010727286350311038),
 (17, 0.0),
 (18, 0.0),
 (19, 0.058321349267840134),
 (20, 0.0),
 (21, 0.0),
 (22, 0.0),
 (23, 0.0),
 (24, 0.0030136767341105873),
 (25, 1.0),
 (26, 0.0),
 (27, 0.035917907722692974),
 (28, 0.005803448123074126),
 (29, 0.017798015596744536),
 (30, 0.012912217205854762),
 (31, 0.011014776597901378),
 (32, 0.010369537913084386),
 (33, 0.017063631264570202),
 (34, 0.02704944297060011),
 (35, 0.0),
 (36, 0.0),
 (37, 0.049954284113239336),
 (38, 0.01143873602257278),
 (39, 0.0),
 (40, 0.0),
 (41, 0.005589882450477922),
 (42, 0.0),
 (43, 0.005898442690409995),
 (44, 0.0138097337431048

In [23]:
len(similarity_score)

4803

In [24]:
# sorting the movies based on similarity score

sorted_similar_movies = sorted(similarity_score,key= lambda x:x[1], reverse=True)
sorted_similar_movies

[(25, 1.0),
 (593, 0.23007271072115248),
 (454, 0.1658990773043873),
 (1081, 0.1603512882331143),
 (2011, 0.13803443166572782),
 (3631, 0.1333839815786052),
 (4133, 0.13183140853614747),
 (656, 0.1260624612880309),
 (1985, 0.12349759621747015),
 (2036, 0.12348320550112098),
 (316, 0.11879207685844574),
 (142, 0.11838847957751894),
 (2449, 0.11814094328522082),
 (846, 0.11610782635112266),
 (49, 0.11503079796465221),
 (765, 0.11360426444934431),
 (439, 0.11317975752102442),
 (2245, 0.11276113457467148),
 (720, 0.10752741389011593),
 (1362, 0.10696934437645385),
 (1629, 0.10631964670577464),
 (1500, 0.10616709183973469),
 (972, 0.10565080846258224),
 (622, 0.10374165072398073),
 (2955, 0.10279003651035283),
 (2175, 0.10248148361918039),
 (250, 0.10070176966728618),
 (4231, 0.09751550112290146),
 (1089, 0.09711335392784962),
 (2008, 0.09668793526881608),
 (961, 0.09638190705612416),
 (351, 0.09601987660900405),
 (3533, 0.09505757385996452),
 (249, 0.09498018720867243),
 (176, 0.0947276913

In [25]:
# print the name of best similar movies based on index

print("Movies suggested for you: \n")

i=1

for movie in sorted_similar_movies:
    index = movie[0]
    title_from_index=mov_data[mov_data.index==index]['title'].values[0]
    if i<20:
        print(i,'.', title_from_index)
        i+=1
    

Movies suggested for you: 

1 . Titanic
2 . The Dilemma
3 . The Day the Earth Stood Still
4 . Revolutionary Road
5 . Cheri
6 . Heavenly Creatures
7 . Plush
8 . Primary Colors
9 . The Bridge of San Luis Rey
10 . Saving Silverman
11 . Gangs of New York
12 . Flushed Away
13 . Sense and Sensibility
14 . Stuck on You
15 . The Great Gatsby
16 . Almost Famous
17 . Shutter Island
18 . The Brothers Bloom
19 . Contagion


# Movie recommendation system

In [26]:
movie_name=input("Enter the fav movie name : ")

list_all_titles=mov_data['title'].to_list()

find_match = difflib.get_close_matches(movie_name,list_all_titles)

close_match=find_match[0]

index_movie=mov_data[mov_data.title == close_match]['index'].values[0]

similarity_score= list(enumerate(similarity[index_movie]))

sorted_similar_movies = sorted(similarity_score,key= lambda x:x[1], reverse=True)

print("Movies suggested for you: \n")

i=1

for movie in sorted_similar_movies:
    index = movie[0]
    title_from_index=mov_data[mov_data.index==index]['title'].values[0]
    if i<20:
        print(i,'.', title_from_index)
        i+=1

Enter the fav movie name : iron man
Movies suggested for you: 

1 . Iron Man
2 . Iron Man 2
3 . Iron Man 3
4 . Avengers: Age of Ultron
5 . The Avengers
6 . Captain America: Civil War
7 . Captain America: The Winter Soldier
8 . Ant-Man
9 . X-Men
10 . Made
11 . X-Men: Apocalypse
12 . X2
13 . The Incredible Hulk
14 . The Helix... Loaded
15 . X-Men: First Class
16 . X-Men: Days of Future Past
17 . Captain America: The First Avenger
18 . Kick-Ass 2
19 . Guardians of the Galaxy
